In [36]:
import joblib
import boto3
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle

s3 = boto3.client('s3')
bucket = 'data-bucket-0f3324' # Зміни на свій бакет
key = 'transformed-data/part-00000-7078c554-9e34-4477-85c0-d77dfe3fdaa5-c000.csv' # Встав сюди назву свого файла

obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(obj['Body'])

encoders = {}

labeled_df = df.copy()

for column in df.columns:
    if labeled_df[column].dtype == 'object': 
        le = LabelEncoder()
        labeled_df[column] = le.fit_transform(labeled_df[column])
        encoders[column] = le

with open('encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)
    
s3.upload_file('encoders.pkl', bucket, 'model/encoders.pkl')
    
labeled_df.to_csv("labeled_data.csv", index = False)

s3.upload_file('labeled_data.csv', bucket, 'labeled_data.csv')

In [37]:
print(df)

          maker   color  reg_year bodytype  runned_miles  engin_size  \
0       Bentley  Silver      2000   Saloon         60000         6.8   
1       Bentley    Grey      2002   Saloon         44000         6.8   
2       Bentley    Blue      2002   Saloon         55000         6.8   
3       Bentley   Green      2003   Saloon         14000         6.8   
4       Bentley    Grey      2003   Saloon         61652         6.8   
...         ...     ...       ...      ...           ...         ...   
255893    Volvo    Grey      2008   Estate        140000         2.0   
255894    Volvo    Blue      2007   Estate        158000         2.4   
255895    Volvo  Silver      2009   Estate         94000         2.4   
255896    Volvo  Silver      2004   Estate        111000         2.4   
255897    Volvo    Blue      2009   Estate        107000         2.0   

          gearbox fuel_type  price  seat_num  door_num  
0       Automatic    Petrol  21500         5         4  
1       Automatic    

In [38]:
print(labeled_df)

        maker  color  reg_year  bodytype  runned_miles  engin_size  gearbox  \
0           5     18      2000        14         60000         6.8        0   
1           5      8      2002        14         44000         6.8        0   
2           5      2      2002        14         55000         6.8        0   
3           5      7      2003        14         14000         6.8        0   
4           5      8      2003        14         61652         6.8        0   
...       ...    ...       ...       ...           ...         ...      ...   
255893     63      8      2008         6        140000         2.0        1   
255894     63      2      2007         6        158000         2.4        0   
255895     63     18      2009         6         94000         2.4        0   
255896     63     18      2004         6        111000         2.4        0   
255897     63      2      2009         6        107000         2.0        1   

        fuel_type  price  seat_num  door_num  
0   

In [3]:
%%writefile script.py
import subprocess
import sys

# Function to install packages
def install_packages():
    packages = [
        "s3fs",
        "scikit-learn",
        "joblib",
        "boto3",
        "numpy",
        "pandas",
        "tqdm"
    ]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import s3fs
import json
from tqdm import tqdm

import pickle
import os
import json
import joblib
import pandas as pd
from io import StringIO

def input_fn(request_body, request_content_type):
    if request_content_type == "application/json":
        try:
            input_json = json.loads([request_body][0])
            df = pd.DataFrame([input_json])
            df = df[['maker', 'color', 'reg_year', 'bodytype', 'runned_miles', 'engin_size', 'gearbox', 'fuel_type', 'seat_num', 'door_num']]
            return df
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'application/json' and send the request!!"""

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))

    with open(os.path.join(model_dir, 'encoders.pkl'), 'rb') as f:
        encoders = pickle.load(f)

    return clf, encoders

def predict_fn(input_data, model_and_encoders):
    model, encoders = model_and_encoders
    
    if isinstance(input_data, pd.DataFrame):
        
        for column in input_data.columns:
            if column in encoders:  
                le = encoders[column]
                input_data[column] = le.transform(input_data[column])

        prediction = model.predict(input_data)
        return prediction[0]
    else:
        return input_data
    
    

if __name__ == "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))

    args, _ = parser.parse_known_args()

    args, _ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()

    s3 = boto3.client('s3')
    bucket = 'data-bucket-0f3324' # Зміни на свій бакет
    key = 'labeled_data.csv'

    obj = s3.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(obj['Body'])
    
    print()
    print(df)
    print()

    X = df.drop('price', axis=1)
    y = df['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=args.random_state)

    print("Training RandomForest Model.....")
    print()
    model = RandomForestRegressor(n_estimators=args.n_estimators, random_state=args.random_state, warm_start=True, n_jobs=-1)

    for i in tqdm(range(1, args.n_estimators + 1)):
        model.n_estimators = i
        model.fit(X_train, y_train)
    
    encoder_path = os.path.join(args.model_dir, "encoders.pkl")
    
    s3.download_file(bucket, "model/encoders.pkl", encoder_path)
        
    print("Encoder persisted at " + encoder_path)
    
    
    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    print(f'R²: {r2}')

Overwriting script.py


In [ ]:
# Можеш не виконувати, то для тестів

! python script.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

In [ ]:
from sagemaker.sklearn.estimator import SKLearn
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd


sagemaker_session = sagemaker.Session()

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [5]:
sklearn_estimator.fit(wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-05-17-08-48-45-216


2024-05-17 08:48:45 Starting - Starting the training job...
2024-05-17 08:48:59 Starting - Preparing the instances for training...
2024-05-17 08:49:30 Downloading - Downloading input data...
2024-05-17 08:49:55 Downloading - Downloading the training image...
2024-05-17 08:50:36 Training - Training image download completed. Training in progress..2024-05-17 08:50:39,701 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-17 08:50:39,706 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 08:50:39,749 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-17 08:50:39,934 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 08:50:39,947 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 08:50:39,960 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-17 08:

In [6]:
sm_boto3 = boto3.client("sagemaker")

sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2024-05-17 08:53:19 Starting - Preparing the instances for training
2024-05-17 08:53:19 Downloading - Downloading the training image
2024-05-17 08:53:19 Training - Training image download completed. Training in progress.
2024-05-17 08:53:19 Uploading - Uploading generated training model
2024-05-17 08:53:19 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-358222979241/RF-custom-sklearn-2024-05-17-08-48-45-216/output/model.tar.gz


In [7]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [8]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2024-05-17-08-54-05


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-05-17-08-54-04
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2024-05-17-08-54-05
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2024-05-17-08-54-05


-------!

In [39]:
import json
import pandas as pd
import joblib
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name=endpoint_name)

random_row = df.sample(n=1)

price = random_row['price'].values[0]

row_dict = random_row.drop(columns=['price']).to_dict(orient='records')[0]

payload = json.dumps(row_dict, ensure_ascii=False)

print(payload)
print(price)


predict_bytes = predictor.predict(payload, initial_args={'ContentType': 'application/json'})

predict = float(predict_bytes.decode('utf-8'))

print(predict)

{"maker": "Vauxhall", "color": "Blue", "reg_year": 2013, "bodytype": "Coupe", "runned_miles": 26468, "engin_size": 2.0, "gearbox": "Manual", "fuel_type": "Petrol", "seat_num": 5, "door_num": 3}
13795
13672.31
